In [1]:
import re
import boto3
import time
import json, copy
from google import genai

In [11]:
database = 'copilot_demo'
table = 'employees_test'
region = 'us-east-1'
default_pii = ["name", "email", "phone_number", "salary"]
athena_output = "s3://de-copilot-s3/athena-results/"

In [12]:
glue_client = boto3.client('glue', region_name=region)

In [13]:
# metadeta stats of the tables in the glue
def glue_metadata(database, table, region):
    try:
        resp = glue_client.get_table(DatabaseName=database, Name=table)
        t = resp['Table']
        user_defined_pii = []
        schema = []
        for col in t["StorageDescriptor"]["Columns"]:
            is_pk = False
            if 'pk' in (col.get('Comment','')).lower() or 'primary_key' in (col.get('Comment', '')).lower():
                is_pk = True

            is_fk = False
            if 'fk' in (col.get('Comment', '')).lower() or 'foreign_key' in (col.get('Comment', '')).lower():
                is_fk = True    

            if 'pii_column' in (col.get('Comment', '')).lower():
                user_defined_pii.append(col["Name"])
                

            schema.append({
                "name": col["Name"],
                "type": col["Type"],
                "nullable": False if is_pk else True,
                "partition_key": False,
                "primary_key" : is_pk,
                "foregin_key" : is_fk, 
                "comments" : col.get("Comment", "")
            })
  
        for col in t.get("PartitionKeys", []):
            schema.append({
                "name": col["Name"],
                "type": col["Type"],
                "nullable": True,
                "partition_key": True,
                "primary_key" : False,
                "foreign_key" : False,
                "comments" : col.get("Comment", "")  
            })
 
        return ({
            "table_name": t["Name"],
            "database": t["DatabaseName"],
            "schema": schema
        },user_defined_pii)

    except Exception as e:
        print("error in glue_metadata:", str(e))
        return ({}, [])

def build_stats_sql(database, table, schema, pii_list):

    pii_columns = set(c.lower() for c in pii_list)
    selects = []

    row_cnts = f""" 
                SELECT
                'ROW_COUNT' as col_name,
                CAST(COUNT(*) AS VARCHAR) AS min_val,
                NULL as max_val,
                NULL as null_pct,
                NULL as distinct_count
                FROM "{database}"."{table}"
                """.strip()
    selects.append(row_cnts)

    for col in schema:
        col_name = col["name"]
        col_lower = col_name.lower()

        if col_lower in pii_columns:
            continue

        s = f"""
            SELECT
            '{col_name}' AS col_name,
            CAST(MIN("{col_name}") AS VARCHAR) AS min_val,
            CAST(MAX("{col_name}") AS VARCHAR) AS max_val,
            AVG(CASE WHEN "{col_name}" IS NULL THEN 1.0 ELSE 0 END) AS null_pct,
            APPROX_DISTINCT("{col_name}") AS distinct_count
            FROM "{database}"."{table}"
            """.strip()

        selects.append(s)

    if not selects:
        return None

    sql = "\nUNION ALL\n".join(selects)
    return sql


def athena_setup(database,region,query,s3_bucket):
    try:
        athena_client = boto3.client('athena',region_name=region)

        response = athena_client.start_query_execution(QueryString=query, QueryExecutionContext = {'Database' : database},
        ResultConfiguration = {'OutputLocation':s3_bucket})

        id = response['QueryExecutionId'] # generated by athena as a ticket number
        
        while True:
            stats = athena_client.get_query_execution(QueryExecutionId=id)
            status = stats['QueryExecution']['Status']['State']

            if status =='SUCCEEDED':
                break 
            elif status in ['FAILED', 'CANCELLED']:
                reason = stats['QueryExecution']['Status'].get('StateChangeReason', 'Unknown')
                raise Exception(f"Query Failed: {reason}")

            time.sleep(5)

        return athena_client.get_query_results(QueryExecutionId=id)

    except Exception as e:
        print("error in run_athena_query:", str(e))
        return None 

# get the column value stats, runs on athena
def get_athena_data(database, table, region, schema, pii_list, s3_bucket):
    try:
        sql = build_stats_sql(database, table, schema, pii_list)

        if not sql:
            print("no non-PII columns found for stats")
            return {}

        query_out = athena_setup(database, region, sql, s3_bucket)

        if not query_out:
            return {}

        rows = query_out["ResultSet"]["Rows"]
        col_stats = parse_stats_rows(rows)


        return col_stats

    except Exception as e:
        print("error in get_athena_data:", str(e))
        return {}

def parse_stats_rows(rows):
    data_rows = rows[1:]

    stats = {'ROW_COUNT' : 0}

    for r in data_rows:
        vals = [c.get("VarCharValue") for c in r["Data"]]


        col_name = vals[0]
        min_val = vals[1]
        if col_name == 'ROW_COUNT':
            stats['ROW_COUNT'] = min_val if min_val else 0
            continue

        max_val = vals[2]
        null_pct = vals[3]
        distinct = vals[4]

        try:
            null_pct = float(null_pct) if null_pct is not None else None
        except:
            pass

        try:
            distinct = int(distinct) if distinct is not None else None
        except:
            pass

        stats[col_name] = {
            "min": min_val,
            "max": max_val,
            "null_pct": null_pct,
            "distinct_count": distinct
        }

    return stats

    

# remove the pii information in the column stats from athena
def filter_pii(col_stats, pii_list):
    pii_columns = set(c.lower() for c in pii_list)

    cleaned_stats = {}
    for col_name, vals in col_stats.items():
        if col_name.lower() not in pii_columns:
            cleaned_stats[col_name] = vals

    return cleaned_stats

ddl_obj, user_defined_pii = glue_metadata(database, table, region)
if not ddl_obj:
    print('No DDL')

if not user_defined_pii:
    user_defined_pii = default_pii

schema = ddl_obj.get("schema", [])
col_stats = get_athena_data(database, table, region, schema, user_defined_pii, athena_output)

clean_stats = filter_pii(col_stats, user_defined_pii)

row_count = clean_stats.get("ROW_COUNT", 0)
column_stats = {k: v for k, v in clean_stats.items() if k != "ROW_COUNT"}

ddl_obj["row_count"] = row_count
ddl_obj["column_stats"] = column_stats

print(json.dumps(ddl_obj, indent=2))



{
  "table_name": "employees_test",
  "database": "copilot_demo",
  "schema": [
    {
      "name": "emp_id",
      "type": "int",
      "nullable": true,
      "partition_key": false,
      "primary_key": false,
      "foregin_key": false,
      "comments": ""
    },
    {
      "name": "name",
      "type": "string",
      "nullable": true,
      "partition_key": false,
      "primary_key": false,
      "foregin_key": false,
      "comments": ""
    },
    {
      "name": "salary",
      "type": "double",
      "nullable": true,
      "partition_key": false,
      "primary_key": false,
      "foregin_key": false,
      "comments": ""
    },
    {
      "name": "department",
      "type": "string",
      "nullable": true,
      "partition_key": false,
      "primary_key": false,
      "foregin_key": false,
      "comments": ""
    },
    {
      "name": "joining_date",
      "type": "date",
      "nullable": true,
      "partition_key": false,
      "primary_key": false,
      "foregi

In [16]:
payload = """
Pay Attention, Do not hallucinate, only work on what is there in the below and think deep for all the edge cases for the below requirements.
Do NOT assume any properties (file format, update frequency, row counts, S3 location, etc.) that are not explicitly present in the JSON. If unknown, either omit or mark as "unknown".

You are a Senior Data Engineer Copilot specializing in **AWS Glue and Athena**.

You will receive a schema object extracted directly from the **AWS Glue Data Catalog**.
Your goal is to generate:
- A Data Quality Contract (pre-load rules with Spark SQL expressions),
- Post-Load Tests (Athena SQL),
- Confluence-style documentation for the table,
- And a separate "suggestions" section with extra OPTIONAL rules for human review.

The JSON object will look like this (shape, not exact values):

{ddl_obj}

Where:
- table_name: name of the table
- database: database / schema name
- schema: list of columns, each like:
  - name: column name
  - type: data type (string, int, double, date, etc.)
  - nullable: true / false
  - partition_key: true / false
  - primary_key: true / false
  - foreign_key: true / false
  - comments: free text with business meaning (e.g. "cannot be null", "PII column", "FK : references employees_test(name)")
- column_stats: ONLY for NON-PII columns, something like:
  - ROW_COUNT: "1234" (stringified)
  - For each non-PII column:
      - min, max, null_pct, distinct_count, top_values
- constraints: optional list of primary keys, unique keys, check constraints (may be missing)
- job_summary: includes inputs, filters, and grain if available
- rule_type MUST be exactly one of the allowed values below.
  Do NOT invent new rule_type names. If unsure, choose the closest one:
  [not_null, not_empty, min, max, allowed_values, regex, pk, fk, check_constraint, custom_sql]

IMPORTANT PRIVACY RULES:
- PII columns (name, email, phone_number, salary, ssn, credit_card, and any column whose comment suggests PII) appear in "schema"
  but their stats and values are NOT provided.
- For PII columns:
  - You MAY define structural rules: not_null, not_empty, length, regex.
  - You MUST NOT include any concrete example values (no fake SSNs, emails, phones, names, etc.).
  - Only describe patterns, like "must be 9 digits", "must match email format".
- For non-PII columns:
  - You MAY use column_stats to propose ranges and allowed_values.
  - Still avoid writing specific sample values in descriptions; talk about rules, not sample data.

----------------------------------------
HOW TO USE COMMENTS
----------------------------------------

You MUST carefully read and interpret the "comments" field for each column. Comments can override or refine behaviour that is not obvious from the "nullable" flag or type alone.

Examples:
- If nullable = true but comments contain phrases like:
  - "cannot be null", "must not be null", "always present", "required field"
  Then you MUST treat this as a business rule: generate a not_null rule and a spark_exp that fails on NULL.

- If comments describe business constraints (e.g. "total number of employees present in this company", "must be positive", "status is one of ACTIVE, INACTIVE"):
  - You should translate this into appropriate rules: min/max, allowed_values, or custom_sql.

- If comments describe relationships (e.g. "FK : references employees_test(name)"):
  - You should generate an fk rule (and a suggested SQL test) that checks referential integrity at the table level.

- If comments mention "PII", "sensitive", or "confidential":
  - Treat the column as PII for purposes of privacy constraints.

Comments are HIGH PRIORITY business knowledge and can override "nullable" if there is a conflict and any other relavant conditions. 
If nullable = true but comment says "cannot be null", you MUST enforce not_null in rules and spark_exp.

----------------------------------------
THINKING / COVERAGE REQUIREMENTS
----------------------------------------

You must think column-by-column and constraint-by-constraint.
Do not skip any column.

For coverage:
- Every column in "schema" (except purely technical partition columns if clearly labeled as such) MUST appear in at least one rule in data_quality.rules.
- Do NOT skip columns just because nullable = true.
- Comments MUST be considered when deciding whether the column should be treated as logically required, even if nullable = true in the physical schema.

For every column in "schema":

1. COMPLETENESS

   IMPORTANT:
   - Normally:
     - If nullable = true, you SHOULD NOT create a not_null rule.
   - However, COMMENTS CAN OVERRIDE NULLABILITY:
     - If comments clearly say "cannot be null", "must not be null", "always present", or similar:
       -> You MUST treat this column as logically NOT NULL and generate a not_null rule and enforce non-null in spark_exp.
   - Nullable columns must allow NULL in all spark_exp expressions unless comments explicitly say otherwise.
   - Never generate contradictory rules for any column.
     Examples of contradictions:
       * nullable = true and comments are silent about nulls, but you generate not_null.
       * comments say "optional" but you enforce not_null.

   - If nullable = false OR comments indicate "cannot be null" → always generate a not_null rule.
   - For ALL string-like columns (string, varchar, char), regardless of nullable:
       * Always generate a not_empty-style rule:
         - If column is logically nullable (nullable = true and comments do NOT forbid nulls):
           spark_exp should be "column IS NULL OR length(trim(column)) > 0".
         - If column is logically NOT NULL (nullable = false OR comments say cannot be null):
           spark_exp should be "column IS NOT NULL AND length(trim(column)) > 0".

2. VALIDITY

   Use the combination of:
   - column name
   - data type
   - comments
   - column_stats (only non-PII)
   - constraints (CHECK, PK, UNIQUE)
   to infer validity rules such as:
     * Numeric columns >= 0 unless obviously not applicable.
     * head_count or similar metrics should be >= 0.
     * String columns with stable lengths → infer min_length / max_length or regex.
     * Year/date columns must not be in the future.
     * Codes (country_code, department, status) with low cardinality should use allowed_values.
     * Comments like "there will be no company without a name" → treat as logically not_null + not_empty.

3. RANGE RULES (NON-PII ONLY)

   Use column_stats[min, max, distinct_count, null_pct].
   - Create soft WARNING rules with a 20–25% buffer around min/max or p95 if present.
   - For example, if min = 1 and max = 287, you might allow [0.8 * min, 1.2 * max] as a WARNING range, if that makes sense for the domain.

4. ALLOWED VALUES (NON-PII ONLY)

   If distinct_count is small (< 50) AND stable → generate allowed_values.
   - You should not list every value if there are too many; only when distinct_count is small.
   - Do not echo literal values if they look like sensitive info; use them mainly when they are clearly categorical codes or domains.

5. PII COLUMNS

   - PII columns appear in schema but have NO column_stats.
   - For these columns you MUST generate:
       * not_null (if nullable = false or comments say cannot be null).
       * not_empty for strings (with logic depending on nullability as above).
       * regex or fixed-length patterns inferred ONLY from schema + column name + comments.
   - NEVER include example email or SSN values. Only describe patterns.

6. CROSS-COLUMN LOGIC (IF OBVIOUS)

   Examples:
   - If year/date columns exist → ensure year <= current_date.
   - If comments suggest referential links (e.g. "FK : references employees_test(name)") → generate fk rules and a suggested SQL test that checks existence in the parent table.
   - If joining_date and resign_date exist → resign_date >= joining_date.
   - If head_count > 0 then company_name should not be null or empty, etc.

7. TABLE-LEVEL RULES

   - If constraints or comments imply a primary key → include uniqueness rule (rule_type = pk at __TABLE__ level).
   - Add table-level rule: row_count > 0 (based on ROW_COUNT in column_stats if present).

8. Data Quality (pre-load PySpark)

   For each rule, you MUST output "spark_exp" using **Spark SQL syntax only**, not PySpark API.
   spark_exp MUST be a SQL expression that can be passed directly into:

       df.filter(expr(spark_exp))

   Examples of valid spark_exp:
     - "salary >= 0"
     - "salary IS NULL OR salary >= 0"
     - "name IS NULL OR length(trim(name)) > 0"
     - "joining_date IS NULL OR joining_date <= current_date()"
     - "head_count IS NULL OR head_count >= 0"

   Examples of INVALID spark_exp (do NOT generate these):
     - col('salary') >= 0
     - F.col("name").isNull()
     - dataframe.count() > 0
     - salary.notNull()
     - any reference to a DataFrame variable

   The semantic convention is:
   - spark_exp should evaluate to TRUE for valid rows.
   - The pipeline will typically use NOT (spark_exp) to find invalid rows.

9. TEST COVERAGE (Post-load SQL)

   You must generate SQL tests for:
       * uniqueness of PK/grain (if applicable).
       * null checks on required columns (including where comments indicate "cannot be null").
       * each CHECK-style constraint or strong business rule you infer.
       * future-date violations.
       * allowed_values validation (for low-cardinality columns).
       * numeric range violations.

After generating rules and tests, REVIEW:
- Did you include ALL columns?
- Did you cover ALL non-nullable and logically non-null columns (including comment-based ones)?
- Did you enforce ALL constraints and clear business rules from comments?
- Did you create BOTH rules AND tests?

----------------------------------------
SUGGESTIONS SECTION (OPTIONAL / OUT-OF-THE-BOX)
----------------------------------------

After you finish generating "data_quality", "tests", and "docs_markdown" (strictly grounded in the input JSON and the rules above), you MUST generate a separate section called "suggestions".

Rules for "suggestions":
- "suggestions" is ONLY for human review. The pipeline does NOT automatically enforce anything from this section.
- In "suggestions" you MAY go beyond the explicit metadata and comments, as long as:
  - You do NOT contradict the schema, data types, or explicit comments.
  - You stay within realistic, common data-engineering best practices.
- Good examples of suggestions:
  - Extra data quality rules that are common in real systems (e.g. latency checks, distribution checks).
  - Additional business-rule candidates that a human could consider.
  - PII / governance hints (e.g. require masking in downstream layers).
  - Suggestions for more detailed allowed_values or cross-table checks.

Each suggestion MUST include:
- A high-level "description" of the suggestion.
- "type": must be exactly "SUGGESTION_ONLY".
- "area": one of [business_rule, dq_rule, pii, governance, performance, schema_design].
- A "rule" object with the SAME structure as a normal rule, plus an optional "sql_test":
  - column, rule_type, condition, severity, action, description, spark_exp, sql_test, notes.

"data_quality" and "tests" MUST remain strictly grounded in the actual JSON and comments.
All speculative or optional ideas MUST go only into "suggestions".

----------------------------------------
OUTPUT FORMAT (MUST BE VALID JSON)
----------------------------------------

Important:
- In the final JSON, the set of column names used in data_quality.rules (excluding "__TABLE__") MUST match the set of column names in "schema" (case-insensitive). Do not omit any columns.
- Coverage requirement does NOT override the schema and comments.
- If a column is nullable, you may generate "if present, must..." rules (e.g., not_empty with NULL allowed).
- If comments explicitly indicate "cannot be null" or equivalent, you MUST treat the column as logically non-null and enforce not_null in both rules and spark_exp.

Return ONLY valid JSON in this exact structure (no extra comments):

{
  "data_quality": {
    "rules": [
      {
        "column": "col_name_or__TABLE__for_table_level",
        "rule_type": "not_null | not_empty | min | max | allowed_values | regex | pk | fk | check_constraint | custom_sql",
        "condition": "value / list / SQL expression / description string",
        "severity": "ERROR | WARNING",
        "action": "FAIL_JOB | DROP_ROW | WARN",
        "description": "Short reasoning for the rule (no concrete example values).",
        "spark_exp": "Spark SQL boolean expression that returns TRUE for valid rows and can be passed directly to pyspark.sql.functions.expr(). It MUST NOT reference any DataFrame variable and MUST NOT call actions like count(), groupBy(), collect(), etc. Examples: \"salary IS NULL OR salary >= 0\", \"name IS NULL OR length(trim(name)) > 0\", \"joining_date IS NULL OR joining_date <= current_date()\"."
      }
    ]
  },
  "tests": [
    {
      "name": "test_name",
      "sql": "SELECT ...",
      "description": "What this test validates."
    }
  ],
  "docs_markdown": "# Table Documentation\n...",
  "suggestions": [
    {
      "area": "business_rule | dq_rule | pii | governance | performance | schema_design",
      "type": "SUGGESTION_ONLY",
      "description": "Human-readable suggestion. Do not treat this as enforced logic.",
      "rule": {
        "column": "col_name_or__TABLE__for_table_level",
        "rule_type": "not_null | not_empty | min | max | allowed_values | regex | pk | fk | check_constraint | custom_sql",
        "condition": "value / list / SQL expression / description string",
        "severity": "ERROR | WARNING",
        "action": "FAIL_JOB | DROP_ROW | WARN",
        "description": "Reasoning for this suggested rule (no concrete sample values).",
        "spark_exp": "Spark SQL boolean expression, same constraints as data_quality.rules.spark_exp.",
        "sql_test": "SELECT ...",
        "notes": "Optional notes. Only apply this after human review."
      }
    }
  ]
}

Do NOT include anything outside this JSON object.

"""


In [19]:
payload = payload.replace("{ddl_obj}", json.dumps(ddl_obj, indent=2))

In [21]:
client = genai.Client()
response = client.models.generate_content(model='gemini-2.5-pro', contents=payload)
print(response.text)

```json
{
  "data_quality": {
    "rules": [
      {
        "column": "__TABLE__",
        "rule_type": "pk",
        "condition": "emp_id",
        "severity": "ERROR",
        "action": "FAIL_JOB",
        "description": "The 'emp_id' column must be unique across all rows. Column stats indicate there are currently duplicate values, which is a critical quality issue.",
        "spark_exp": "null"
      },
      {
        "column": "__TABLE__",
        "rule_type": "custom_sql",
        "condition": "row_count > 0",
        "severity": "ERROR",
        "action": "FAIL_JOB",
        "description": "The table must contain at least one row to be considered valid for processing.",
        "spark_exp": "null"
      },
      {
        "column": "emp_id",
        "rule_type": "not_null",
        "condition": null,
        "severity": "ERROR",
        "action": "FAIL_JOB",
        "description": "Employee ID is a required identifier and cannot be null, based on business context and observed 0

# Push to S3

In [11]:
contract_json_path = "s3://de-copilot-s3/contracts/"
s3_client = boto3.client('s3',region_name=region)

In [12]:
clean_json_str = re.sub(r"```json\n|\n```", "", response.text).strip()

In [13]:
cleaned_json = json.loads(clean_json_str)

In [14]:
s3_client.put_object(Bucket='de-copilot-s3',Key=f'contracts/{table}.json',Body=json.dumps(cleaned_json,indent=2),ContentType='application/json')

{'ResponseMetadata': {'RequestId': 'MAY7KSGK88MB18RK',
  'HostId': 'oZ+X5mZ309qKCKN8nAE6DDziYBla5fBjNq0L/boUIj0EyUuNIqx862uw2uhEkh55VtH6PudYUH4fhR83r/D9t/A1S9BoGXZv',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'oZ+X5mZ309qKCKN8nAE6DDziYBla5fBjNq0L/boUIj0EyUuNIqx862uw2uhEkh55VtH6PudYUH4fhR83r/D9t/A1S9BoGXZv',
   'x-amz-request-id': 'MAY7KSGK88MB18RK',
   'date': 'Sun, 23 Nov 2025 18:58:33 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"b185c99ce1de48a934ecb2165038253f"',
   'x-amz-checksum-crc32': '8+l7aA==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"b185c99ce1de48a934ecb2165038253f"',
 'ChecksumCRC32': '8+l7aA==',
 'ChecksumType': 'FULL_OBJECT',
 'ServerSideEncryption': 'AES256'}

In [58]:
ddl_obj

{'table_name': 'employees_test',
 'database': 'copilot_demo',
 'schema': [{'name': 'emp_id',
   'type': 'int',
   'nullable': False,
   'partition_key': False,
   'primary_key': True,
   'foregin_key': False,
   'comments': 'primary_key'},
  {'name': 'name',
   'type': 'string',
   'nullable': True,
   'partition_key': False,
   'primary_key': False,
   'foregin_key': False,
   'comments': ''},
  {'name': 'salary',
   'type': 'double',
   'nullable': True,
   'partition_key': False,
   'primary_key': False,
   'foregin_key': False,
   'comments': ''},
  {'name': 'department',
   'type': 'string',
   'nullable': True,
   'partition_key': False,
   'primary_key': False,
   'foregin_key': False,
   'comments': ''},
  {'name': 'joining_date',
   'type': 'date',
   'nullable': True,
   'partition_key': False,
   'primary_key': False,
   'foregin_key': False,
   'comments': 'cannot be null'}],
 'column_stats': {'ROW_COUNT': '261',
  'joining_date': {'min': '2011-01-17',
   'max': '2023-11-30